## Data Cleaning 

In [1]:
import csv

In [2]:
#used this to look at the dat files to see if they loaded properly
"""
with open('ml-1m/ratings.dat') as fn:
    for line in fn:
        print(line)
"""

"\nwith open('ml-1m/ratings.dat') as fn:\n    for line in fn:\n        print(line)\n"

In [2]:
with open('ml-1m/movies.dat') as dat_file, open('ml-1m/movies.csv', 'w') as csv_file:
    ####
    #this was used to headers into the csv as a result of misreading the directions
    #writer = csv.DictWriter(csv_file, fieldnames=['MovieID', 'Title', 'Year', 'Genre','Genre','Genre', 'Genre'])
    #writer.writeheader()
    #####
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        if "::" in line:
            row = line.strip()
            row = row.split("::")
            row = "|".join(row)
            row = row.split('|')
            row = row[:3]
        for i in range(len(row)):
            if ',' in row[1]:
                row[1] = row[1].replace(',','')
        
            
            """
        #this loop was use to make the csv much cleaner by separating the 
        #title and year 
        for i in range(len(row)):
            if '(' in row[i]:
                row[i] = row[i].strip()
                row[i] = row[i].split("(")
                row[i] = ")".join(row[i])
                row[i] = row[i].split(')')
                row[i] = row[i][:-1]
                titled_row = row[i][0]
                year_row = row[i][1]
        # created copy of list 'row' as "new_row"
        #This solution still updates the a list in place. 
        #To avoid it, change the order of lines: new_row=row[:]; row.insert(2,3)
        del row[1]
        for j in range(len(row)):
            new_row = row[:]
            new_row.insert(1,titled_row)
            new_row.insert(2, year_row)
            """
        csv.writer(csv_file).writerow(row)  

In [179]:
#this function is use to avoid DRY 
#since the other files have similiar formats
#def convert_dat2csv(file_name, header_list):
    #with open('ml-1m/{}'.format(file_name)) as file_dat, open('ml-1m/{}.csv'.format(file_name), 'w',newline ='') as file_csv:
def convert_dat2csv(file_name):
    with open('ml-1m/{}'.format(file_name)) as file_dat, open('ml-1m/{}.csv'.format(file_name), 'w') as file_csv:
        #writer = csv.DictWriter(file_csv, fieldnames=header_list)
        #writer.writeheader()
        csv_writer = csv.writer(file_csv)

        for line in file_dat:
            if "::" in line:
                row = line.strip()
                row = row.split("::")
            for i in range(len(row)):
                if '-' in row[4]:
                    row[4] = row[4].replace('-','')
                    
                               
            csv.writer(file_csv).writerow(row)  

In [185]:
def convert_dat_to_csv(file_name):
    with open('ml-1m/{}'.format(file_name)) as file_dat, open('ml-1m/{}.csv'.format(file_name), 'w') as file_csv:
        #writer = csv.DictWriter(file_csv, fieldnames=header_list)
        #writer.writeheader()
        csv_writer = csv.writer(file_csv)

        for line in file_dat:
            if "::" in line:
                row = line.strip()
                row = row.split("::")
                    
                               
            csv.writer(file_csv).writerow(row)  

In [148]:
r_header = ['UserID', 'MovieID', 'Rating','Timestamp']
u_header = ['UserID','Gender', 'Age', 'Occupation', 'ZipCode']

In [183]:
file_dat1 = 'users.dat'
file_dat2 ='ratings.dat' 

In [182]:
convert_dat2csv(file_dat1)

In [186]:
convert_dat_to_csv(file_dat2)

In [187]:
#used this to look at the csv files to see how to place the header columns
"""
with open('ml-1m/ratings.dat.csv') as fn:
    for line in fn:
        print(line)
"""

"\nwith open('ml-1m/ratings.dat.csv') as fn:\n    for line in fn:\n        print(line)\n"

In [188]:
def tidy_csv(csv_dirty,csv_clean):
    with open('ml-1m/{}'.format(csv_dirty)) as input_csv, open('ml-1m/{}'.format(csv_clean), 'w', newline='') as output:
        writer = csv.writer(output)
        for row in csv.reader(input_csv):
            if any(field.strip() for field in row):
                writer.writerow(row)

In [189]:
tidy_csv('movies.csv','clean_movies.csv')

In [190]:
tidy_csv('users.dat.csv','clean_users.csv')

In [191]:
tidy_csv('ratings.dat.csv','clean_ratings.csv')

## Importing CSV into SQL

In [1]:
import csv, psycopg2

In [2]:
# Creating a new database by changing the name within the quotes
conn = psycopg2.connect(host='localhost',
                       dbname='postgres',
                       user='postgres',
                       password='Scizor212')  

In [3]:
#create a cursor object 
#cursor object is used to interact with the database
c = conn.cursor()

In [7]:
#c.execute('DROP TABLE If Exists "Movies";')

InternalError: current transaction is aborted, commands ignored until end of transaction block


`CREATE TABLE` command with the columns in the same order as the CSV file and their respective types. Similar to running a `SELECT` query, we will write the command as a string and pass it to the `execute()` method. Here’s how it would look for this table:

In [195]:
c.execute("""CREATE TABLE Movies
(MovieID Integer Primary Key ,
Title Text,
Genre Text)""")

In [196]:
c.execute('''CREATE TABLE Users 
(UserID Integer Primary Key, 
Gender Text, 
Age Integer, 
Occupation Integer, 
ZipCode Integer);''')

In [197]:
c.execute('''Create Table Ratings 
(UserID Integer,
MovieID Integer,
Rating Integer,
Timestamp Integer)''')

In [198]:
with open('ml-1m/clean_movies.csv', 'r') as f:
# Notice that we don't need the `csv` module.
#next(f) # Skip the header row.
    c.copy_from(f, 'Movies', sep=',')
    conn.commit()

In [199]:
with open('ml-1m/clean_users.csv', 'r') as u_csv:
    c.copy_from(u_csv, 'Users', sep=',')
    conn.commit()

In [200]:
with open('ml-1m/clean_ratings.csv', 'r') as r_csv:
    c.copy_from(r_csv, 'Ratings', sep=',')
    conn.commit()

## SQL Joins

In [4]:
#this is to test that all the data was transferred to their assign tables
postgresql_select1 = 'Select * From Movies'
postgresql_select2 = 'Select * From Users'
postgresql_select3 = 'Select * From Ratings'
cur = conn.cursor()

In [5]:
cur.execute(postgresql_select1)

In [6]:
movies_test = cur.fetchmany(10)
print(movies_test)

[(1, 'Toy Story (1995)', 'Animation'), (2, 'Jumanji (1995)', 'Adventure'), (3, 'Grumpier Old Men (1995)', 'Comedy'), (4, 'Waiting to Exhale (1995)', 'Comedy'), (5, 'Father of the Bride Part II (1995)', 'Comedy'), (6, 'Heat (1995)', 'Action'), (7, 'Sabrina (1995)', 'Comedy'), (8, 'Tom and Huck (1995)', 'Adventure'), (9, 'Sudden Death (1995)', 'Action'), (10, 'GoldenEye (1995)', 'Action')]


In [7]:
cur.execute(postgresql_select2)

In [8]:
users_test = cur.fetchmany(10)
print(users_test)

[(1, 'F', 1, 10, 48067), (2, 'M', 56, 16, 70072), (3, 'M', 25, 15, 55117), (4, 'M', 45, 7, 2460), (5, 'M', 25, 20, 55455), (6, 'F', 50, 9, 55117), (7, 'M', 35, 1, 6810), (8, 'M', 25, 12, 11413), (9, 'M', 25, 17, 61614), (10, 'F', 35, 1, 95370)]


In [9]:
cur.execute(postgresql_select3)

In [10]:
ratings_test = cur.fetchmany(10)
print(ratings_test)

[(1, 1193, 5, 978300760), (1, 661, 3, 978302109), (1, 914, 3, 978301968), (1, 3408, 4, 978300275), (1, 2355, 5, 978824291), (1, 1197, 3, 978302268), (1, 1287, 5, 978302039), (1, 2804, 5, 978300719), (1, 594, 4, 978302268), (1, 919, 4, 978301368)]


In [8]:
postgresql_select4 = """
Select m.Title,
m.Genre,
u.UserID,
u.Gender,
u.Age, 
u.Occupation,
u.ZipCode,
r.Rating,
r.Timestamp
From Movies m
Inner Join Ratings r On m.MovieID = r.MovieID
Inner Join Users u On r.UserID = u.UserID;"""
cur = conn.cursor()

In [14]:
cur.execute(postgresql_select4)

In [15]:
search_results = cur.fetchmany(10)
print(search_results)

[("One Flew Over the Cuckoo's Nest (1975)", 'Drama', 1, 'F', 1, 10, 48067, 5, 978300760), ('James and the Giant Peach (1996)', 'Animation', 1, 'F', 1, 10, 48067, 3, 978302109), ('My Fair Lady (1964)', 'Musical', 1, 'F', 1, 10, 48067, 3, 978301968), ('Erin Brockovich (2000)', 'Drama', 1, 'F', 1, 10, 48067, 4, 978300275), ("Bug's Life A (1998)", 'Animation', 1, 'F', 1, 10, 48067, 5, 978824291), ('Princess Bride The (1987)', 'Action', 1, 'F', 1, 10, 48067, 3, 978302268), ('Ben-Hur (1959)', 'Action', 1, 'F', 1, 10, 48067, 5, 978302039), ('Christmas Story A (1983)', 'Comedy', 1, 'F', 1, 10, 48067, 5, 978300719), ('Snow White and the Seven Dwarfs (1937)', 'Animation', 1, 'F', 1, 10, 48067, 4, 978302268), ('Wizard of Oz The (1939)', 'Adventure', 1, 'F', 1, 10, 48067, 4, 978301368)]


In [16]:
search_results1 = cur.fetchall()

In [17]:
import json
import collections

In [18]:
# Convert query to objects of key-value pairs
objects_list = []
for row in search_results1:
    d = collections.OrderedDict()
    d['Title'] = row[0]
    d['Genre'] = row[1]
    d['UserID'] = row[2]
    d['Gender'] = row[3]
    d['Age'] = row[4]
    d['Occupation'] = row[5]
    d['ZipCode'] = row[6]
    d['Rating'] = row[7]
    d['Timestamp'] = row[8]
    objects_list.append(d)

In [83]:
#convert the sql query to json
with open('ml-1m/nosql_data.json', 'w') as outfile:
    json.dump(objects_list, outfile, indent=4)

In [21]:
from tinydb import TinyDB, Query, where

tiny_db = TinyDB('ml-1m/ml_data.json')

In [22]:
#I tried to use the 'insert_multiple' and it did not work me 
#So this was the only method that work for me and since it takes too long,
#I just inserted the first 10000 rows to save time 
# This version gets all the rows
#for i in range(len(objects_list)):
#    tiny_db.insert(objects_list)

i = 0
while i < 10001:
    tiny_db.insert(objects_list[i])
    i+=1

## Questions

1. Using the relational database you built, compare M and F average ratings for "Die Hard."

In [24]:
movie_query = """Select m.Title, u.Gender, avg(r.Rating)
from Movies m 
inner join Ratings r on m.MovieID = r.MovieID 
inner join Users u on r.UserID = u.UserID 
where u.Gender = 'F' and m.Title = 'Die Hard (1988)'
group by m.Title, u.Gender;"""
cur = conn.cursor()

In [25]:
cur.execute(movie_query)

In [26]:
search_resultsF = cur.fetchall()
print(search_resultsF)

[('Die Hard (1988)', 'F', Decimal('3.9185667752442997'))]


In [27]:
movie_query = """Select m.Title, u.Gender, avg(r.Rating)
from Movies m 
inner join Ratings r on m.MovieID = r.MovieID 
inner join Users u on r.UserID = u.UserID 
where u.Gender = 'M' and m.Title = 'Die Hard (1988)'
group by m.Title, u.Gender;"""
cur = conn.cursor()

In [28]:
cur.execute(movie_query)

In [29]:
search_resultsM = cur.fetchall()
print(search_resultsM)

[('Die Hard (1988)', 'M', Decimal('4.1677704194260486'))]


2. Do the same comparison with the NoSQL database.

In [30]:
female_dh_set = tiny_db.search( (where('Title').matches('Die Hard')) & (where('Gender').matches('F')) )

In [32]:
dh_avg_f = sum(int(r['Rating']) for r in female_dh_set) / len(female_dh_set)

In [38]:
print('Average: {}'.format(dh_avg_f))

Average: 3.0


In [35]:
male_dh_set = tiny_db.search( (where('Title').matches('Die Hard')) & (where('Gender').matches('M')) )

In [36]:
dh_avg_m = sum(int(r['Rating']) for r in male_dh_set) / len(male_dh_set)

In [39]:
print('Average: {}'.format(dh_avg_m))

Average: 3.769230769230769


3. Do the averages match?

No, but the reason is that I use a smaller portion of the dataset in the TinyDB instead of the full dataset. However, if I were able to use the full dataset I think that both the Sql and NoSql averages would be similiar.

4. What is the age range of female reviewers of "Gone With The Wind?" (Hint: in SQL, you can use a column more than once. Hint 2: There may be built in functions that help.)

In [48]:
def avg_age(movie_title,gender, avg_column):
    female_dh_set = tiny_db.search( (where('Title').matches(movie_title)) & (where('Gender').matches(gender)) )
    dh_avg_f = sum(int(u[avg_column]) for u in female_dh_set) / len(female_dh_set)
    return 'Average: {}'.format(dh_avg_f)

In [49]:
avg_age('Gone with the Wind', 'F', 'Age')

'Average: 36.8'

In [54]:
age_query = """Select m.Title, u.Gender, avg(u.Age)
from Movies m 
inner join Ratings r on m.MovieID = r.MovieID 
inner join Users u on r.UserID = u.UserID 
where u.Gender = 'F' and m.Title = 'Gone with the Wind (1939)'
group by m.Title, u.Gender;"""
cur = conn.cursor()

In [55]:
cur.execute(age_query)

In [56]:
search_results_Age = cur.fetchall()
print(search_results_Age)

[('Gone with the Wind (1939)', 'F', Decimal('32.8956916099773243'))]


## Convert Sql to csv

In [9]:
cur.execute(postgresql_select4)

In [11]:
with open("ml-1m/sql_query_out.csv", "w", newline='') as csv_file:  # Python 3 version    
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow([i[0] for i in cur.description]) # write headers
    csv_writer.writerows(cur)